In [1]:
import numpy as np
import cv2
import sys
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython.display import Image

In [3]:
## Functional API
inputs = keras.Input(shape = (150, 150, 3))
Conv1 = keras.layers.Conv2D(64, kernel_size = 3, activation = 'relu')(inputs)
Conv2 = keras.layers.Conv2D(32, kernel_size = 3, activation = 'relu')(Conv1)
max_pool = keras.layers.MaxPool2D(pool_size = 2)(Conv2)
flatten = keras.layers.Flatten()(max_pool)
dense1 = keras.layers.Dense(120, activation = 'relu')(flatten)
drop_out = keras.layers.Dropout(0.3)(dense1)
dense2 = keras.layers.Dense(32, activation = 'relu')(drop_out)
outputs = keras.layers.Dense(1, activation = 'sigmoid')(dense2)

alz_model = keras.Model(inputs = inputs, outputs = outputs)
alz_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 73, 73, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 170528)            0         
                                                                 
 dense (Dense)               (None, 120)               20463480  
                                                             

In [4]:
import glob

In [17]:
img_path_ad = glob.glob('./Ch_1_fig/fig/data/train/train/ad/*.*') + glob.glob('./Ch_1_fig/fig/data/test/test/ad/*.*')
img_path_normal = glob.glob('./Ch_1_fig/fig/data/train/train/normal/*.*') + glob.glob('./Ch_1_fig/fig/data/test/test/normal/*.*')
img_paths = img_path_ad + img_path_normal

target = np.array([1]*len(img_path_ad) + [0]*len(img_path_normal))
# display(len(img_path_ad))
# display(len(img_paths))
# print(target)
# print(len(target))
# display(img_paths)

imgs = np.zeros((0,150,150,3), np.float32)

for img_path in img_paths:
    img = cv2.imread(img_path)
    img = (img.astype(np.float32))/255.
    img = np.expand_dims(img, axis=0)
    imgs = np.append(imgs, img, axis=0)
    
print(imgs.shape)

(280, 150, 150, 3)


In [18]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split( imgs, target, test_size=0.2, shuffle=True)

print(x_train.shape)
print(y_train.shape)

(224, 150, 150, 3)
(224,)


In [22]:
alz_model.compile(loss='binary_crossentropy',
                 optimizer = 'adam',
                 metrics = ['acc'])

checkpoint = keras.callbacks.ModelCheckpoint(filepath = './alz_model.h5',
                                            save_best_only = True)
early_stop = keras.callbacks.EarlyStopping(patience=7)

In [25]:
history = alz_model.fit(x_train, y_train, epochs=100,
                        batch_size=5,
                       validation_data = (x_val, y_val),
                       callbacks = [checkpoint, early_stop])

Epoch 1/100
45/45 [==============================] - 17s 385ms/step - loss: 11542.6396 - acc: 0.4866 - val_loss: 21.1791 - val_acc: 0.8750
Epoch 2/100
45/45 [==============================] - 17s 372ms/step - loss: 8952.6240 - acc: 0.5045 - val_loss: 2247.8118 - val_acc: 0.6429
Epoch 3/100
45/45 [==============================] - 17s 377ms/step - loss: 6787.8569 - acc: 0.4955 - val_loss: 706.6304 - val_acc: 0.6429
Epoch 4/100
45/45 [==============================] - 16s 353ms/step - loss: 6206.1597 - acc: 0.5000 - val_loss: 1215.0018 - val_acc: 0.6429
Epoch 5/100
45/45 [==============================] - 16s 354ms/step - loss: 3993.4348 - acc: 0.5179 - val_loss: 1658.0736 - val_acc: 0.3571
Epoch 6/100
45/45 [==============================] - 16s 361ms/step - loss: 4488.9741 - acc: 0.4062 - val_loss: 5202.5171 - val_acc: 0.3571
Epoch 7/100
45/45 [==============================] - 16s 356ms/step - loss: 2783.4805 - acc: 0.6027 - val_loss: 2712.4575 - val_acc: 0.3571
Epoch 8/100
45/45 [===